# 微调在ImageNet上预训练的卷积神经网络实现鸟类识别

## 模型介绍 : 
## 1. 修改ResNet-18架构用于鸟类识别，通过将其输出层大小设置为200以适应数据集中的类别数量，其余层使用在ImageNet上预训练得到的网络参数进行初始化。
## 2. 在[CUB-200-2011]( https://data.caltech.edu/records/65de6-vp158)数据集（训练集：80%， 验证集：20%）上从零开始训练新的输出层，并对其余参数使用较小的学习率进行微调
## 3. 通过调整不同的超参数组合（LR:[0.01, 0.001]）(epoch:[10, 15, 20]) (batch_size:[16, 32]),观察在验证集上面的效果，最后找到最好的参数组合
## 4.使用最好的参数组合与从随机初始化的网络参数开始训练得到的参数组合进行对比，观察最后的实验效果

# 

# 模型权重链接： https://pan.baidu.com/s/1a3BqgSVdEmhYPXWSuXyPvQ?pwd=xjhz 提取码: xjhz --来自百度网盘超级会员v6的分享
# 数据集相关链接：https://data.caltech.edu/records/65de6-vp158
# github链接：https://github.com/liuyhoong/-ImageNet-.git 
# 相关代码： git clone https://github.com/liuyhoong/-ImageNet-.git 

# 

# 先导入必须的库

In [1]:
import torch.optim as optim
from torchvision import datasets, transforms
import os
import torch
import torch.nn as nn
import torchvision.models as models
import pandas as pd
import matplotlib.pyplot as plt
import random
from torch.utils.data import DataLoader, random_split

#  选择预训练模型ResNet-18并调整输出层大小为200

In [2]:
# 选择预训练模型ResNet-18
model = models.resnet18(pretrained=True)

# 修改最后一层，全连接层的输入大小保持不变，输出大小设置为200
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)

d:\Anaconda\envs\yolov8\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\yolov8\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\13277/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


# 读取数据集并划分为训练集和验证集

In [4]:
random.seed(42)

data_dir = 'D:/FDU/graduate/研一下/神网/期中作业/CUB_200_2011/CUB_200_2011/images'

# 定义数据转换
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 加载整个数据集
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# 获取类别索引
class_indices = {cls: [] for cls in full_dataset.classes}

# 按类别收集图像索引
for idx, (path, class_idx) in enumerate(full_dataset.imgs):
    class_indices[full_dataset.classes[class_idx]].append(idx)

# 划分训练集和验证集
train_indices = []
val_indices = []

for cls, indices in class_indices.items():
    random.shuffle(indices)
    split = int(0.8 * len(indices))  # 80% 训练，20% 验证
    train_indices.extend(indices[:split])
    val_indices.extend(indices[split:])

# 创建训练集和验证集
train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset, val_indices)

# 应用不同的transform
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# 创建DataLoader
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 输出一些信息以确认
print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")

Total images: 11788
Training images: 9414
Validation images: 2374


# 找到最好的超参数

In [6]:
# 设置 device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 训练代码
learning_rates = [1e-2, 1e-3]
num_epochs_list = [10, 15, 20]
batch_sizes = [16, 32]

for lr in learning_rates:
    for num_epochs in num_epochs_list:
        for batch_size in batch_sizes:
            # 重新初始化数据加载器
            dataloaders = {'train': train_loader, 'val': val_loader}
            
            # 重新初始化模型和优化器
            model = models.resnet18(pretrained=True)
            num_ftrs = model.fc.in_features
            model.fc = nn.Linear(num_ftrs, 200)
            model = model.to(device)
            other_params = [param for name, param in model.named_parameters() if "fc" not in name]
            optimizer = optim.SGD([
    {'params': model.fc.parameters(), 'lr': lr},
    {'params': other_params, 'lr': lr * 0.1}
], momentum=0.9, weight_decay=1e-4)
            
            # 开始训练
            for epoch in range(num_epochs):
                for phase in ['train', 'val']:
                    if phase == 'train':
                        model.train()
                    else:
                        model.eval()
                    
                    running_loss = 0.0
                    running_corrects = 0
                    
                    for inputs, labels in dataloaders[phase]:
                        inputs = inputs.to(device)
                        labels = labels.to(device)
                        
                        optimizer.zero_grad()
                        
                        with torch.set_grad_enabled(phase == 'train'):
                            outputs = model(inputs)
                            _, preds = torch.max(outputs, 1)
                            loss = criterion(outputs, labels)
                            
                            if phase == 'train':
                                loss.backward()
                                optimizer.step()
                        
                        running_loss += loss.item() * inputs.size(0)
                        running_corrects += torch.sum(preds == labels.data)
                    
                    epoch_loss = running_loss / len(dataloaders[phase].dataset)
                    epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
                    
                    print(f'LR: {lr}, Epoch: {epoch}/{num_epochs - 1}, Batch Size: {batch_size}, {phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training completed.")

LR: 0.01, Epoch: 0/9, Batch Size: 16, train Loss: 3.1294 Acc: 0.3187
LR: 0.01, Epoch: 0/9, Batch Size: 16, val Loss: 1.6145 Acc: 0.5826
LR: 0.01, Epoch: 1/9, Batch Size: 16, train Loss: 1.1407 Acc: 0.7167
LR: 0.01, Epoch: 1/9, Batch Size: 16, val Loss: 1.1463 Acc: 0.6967
LR: 0.01, Epoch: 2/9, Batch Size: 16, train Loss: 0.6435 Acc: 0.8421
LR: 0.01, Epoch: 2/9, Batch Size: 16, val Loss: 0.9918 Acc: 0.7279
LR: 0.01, Epoch: 3/9, Batch Size: 16, train Loss: 0.3727 Acc: 0.9220
LR: 0.01, Epoch: 3/9, Batch Size: 16, val Loss: 0.9730 Acc: 0.7279
LR: 0.01, Epoch: 4/9, Batch Size: 16, train Loss: 0.2195 Acc: 0.9638
LR: 0.01, Epoch: 4/9, Batch Size: 16, val Loss: 0.9416 Acc: 0.7397
LR: 0.01, Epoch: 5/9, Batch Size: 16, train Loss: 0.1277 Acc: 0.9869
LR: 0.01, Epoch: 5/9, Batch Size: 16, val Loss: 0.9147 Acc: 0.7447
LR: 0.01, Epoch: 6/9, Batch Size: 16, train Loss: 0.0816 Acc: 0.9954
LR: 0.01, Epoch: 6/9, Batch Size: 16, val Loss: 0.9089 Acc: 0.7511
LR: 0.01, Epoch: 7/9, Batch Size: 16, train Loss

 # 训练为预训练过的模型，训练50轮（轮数太少无法收敛，太多则会过拟合，所以选择50轮）

In [8]:
# 初始化未预训练的ResNet-18模型
model_scratch = models.resnet18(pretrained=False)
num_ftrs_scratch = model_scratch.fc.in_features
model_scratch.fc = nn.Linear(num_ftrs_scratch, 200)
model_scratch = model_scratch.to(device)

# 设置优化器
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=1e-3, momentum=0.9, weight_decay=1e-4)

# 设置损失函数
criterion = nn.CrossEntropyLoss()

num_epochs = 50
for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model_scratch.train()
        else:
            model_scratch.eval()
        
        running_loss = 0.0
        running_corrects = 0
        
        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer_scratch.zero_grad()
            
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model_scratch(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer_scratch.step()
            
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(dataloaders[phase].dataset)
        epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
        
        print(f'Scratch {phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

d:\Anaconda\envs\yolov8\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Anaconda\envs\yolov8\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Scratch train Loss: 5.2785 Acc: 0.0092
Scratch val Loss: 5.1492 Acc: 0.0135
Scratch train Loss: 5.0180 Acc: 0.0241
Scratch val Loss: 4.8945 Acc: 0.0312
Scratch train Loss: 4.7761 Acc: 0.0359
Scratch val Loss: 4.7120 Acc: 0.0447
Scratch train Loss: 4.5866 Acc: 0.0468
Scratch val Loss: 4.5505 Acc: 0.0518
Scratch train Loss: 4.4235 Acc: 0.0637
Scratch val Loss: 4.3879 Acc: 0.0611
Scratch train Loss: 4.2778 Acc: 0.0818
Scratch val Loss: 4.3064 Acc: 0.0775
Scratch train Loss: 4.1449 Acc: 0.0963
Scratch val Loss: 4.2145 Acc: 0.0906
Scratch train Loss: 4.0139 Acc: 0.1167
Scratch val Loss: 4.1565 Acc: 0.0906
Scratch train Loss: 3.8899 Acc: 0.1313
Scratch val Loss: 4.1716 Acc: 0.0952
Scratch train Loss: 3.7733 Acc: 0.1503
Scratch val Loss: 3.9828 Acc: 0.1163
Scratch train Loss: 3.6546 Acc: 0.1656
Scratch val Loss: 3.8321 Acc: 0.1276
Scratch train Loss: 3.5340 Acc: 0.1839
Scratch val Loss: 3.8713 Acc: 0.1306
Scratch train Loss: 3.4159 Acc: 0.1977
Scratch val Loss: 3.6679 Acc: 0.1702
Scratch trai

# 将训练结果使用Tensorboard展示

In [ ]:
import pandas as pd
import tensorflow as tf
import datetime
import os

# 读取数据
train_file_path = 'D:/FDU/graduate/研一下/神网/期中作业/训练集.xlsx'
validation_file_path = 'D:/FDU/graduate/研一下/神网/期中作业/验证集.xlsx'
train_df = pd.read_excel(train_file_path)
validation_df = pd.read_excel(validation_file_path)

# 创建一个日志目录
user_home = os.path.expanduser('~')
log_dir = os.path.join(user_home, "tensorboard_logs", "training_phases_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
os.makedirs(log_dir, exist_ok=True)  # 确保目录存在
writer = tf.summary.create_file_writer(log_dir)

# 分割训练阶段
train_phase_num = 0
validation_phase_num = 0

with writer.as_default():
    # 记录训练集数据
    for i in range(len(train_df)):
        epoch = train_df.loc[i, 'Epoch']
        train_loss = train_df.loc[i, 'Train_Loss']
        train_accuracy = train_df.loc[i, 'Acc']

        # 检测到新的训练阶段
        if epoch == 0 and i != 0:
            train_phase_num += 1
        
        # 写入 TensorBoard 日志
        tf.summary.scalar(f'Train_Phase_{train_phase_num}/loss', train_loss, step=epoch)
        tf.summary.scalar(f'Train_Phase_{train_phase_num}/accuracy', train_accuracy, step=epoch)

    # 记录验证集数据
    for i in range(len(validation_df)):
        epoch = validation_df.loc[i, 'Epoch']
        validation_loss = validation_df.loc[i, 'Val_Loss']
        validation_accuracy = validation_df.loc[i, 'Acc']

        # 检测到新的验证阶段
        if epoch == 0 and i != 0:
            validation_phase_num += 1
        
        # 写入 TensorBoard 日志
        tf.summary.scalar(f'Validation_Phase_{validation_phase_num}/loss', validation_loss, step=epoch)
        tf.summary.scalar(f'Validation_Phase_{validation_phase_num}/accuracy', validation_accuracy, step=epoch)

    writer.flush()

import subprocess
import time

# 设置日志目录
log_dir = os.path.expanduser('~/tensorboard_logs')

# 启动 TensorBoard 的命令
tensorboard_cmd = ['tensorboard', '--logdir', log_dir, '--host', 'localhost', '--port', '6006']

# 使用 subprocess 启动 TensorBoard
tensorboard_proc = subprocess.Popen(tensorboard_cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

time.sleep(5)

# 输出访问 URL
print("TensorBoard 已启动，请访问 http://localhost:6006 查看可视化结果。")

import tensorflow as tf
import pandas as pd
import os

# 加载Excel文件
train_df = pd.read_excel('D:/FDU/graduate/研一下/神网/期中作业/未提前训练的训练集.xlsx')
val_df = pd.read_excel('D:/FDU/graduate/研一下/神网/期中作业/未提前训练的验证集.xlsx')

# 日志目录存在于用户的主目录下
home_dir = os.path.expanduser('~')
log_dir = os.path.join(home_dir, 'tensorboard_logs')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

# 创建日志写入器
train_log_dir = os.path.join(log_dir, 'train')
val_log_dir = os.path.join(log_dir, 'val')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
val_summary_writer = tf.summary.create_file_writer(val_log_dir)

# 从数据框中获取值
train_loss = train_df['Train_Loss'].values
train_acc = train_df['Acc'].values
val_loss = val_df['Val_Loss'].values
val_acc = val_df['Acc'].values

# 写入摘要
with train_summary_writer.as_default():
    for epoch, (loss, acc) in enumerate(zip(train_loss, train_acc)):
        tf.summary.scalar('loss', loss, step=epoch)
        tf.summary.scalar('accuracy', acc, step=epoch)

with val_summary_writer.as_default():
    for epoch, (loss, acc) in enumerate(zip(val_loss, val_acc)):
        tf.summary.scalar('loss', loss, step=epoch)
        tf.summary.scalar('accuracy', acc, step=epoch)

print(f'TensorBoard日志已保存到目录 "{log_dir}"。')

# 使用最优超参数进行训练并与50轮进行对比分析
# 最优超参数为：learning_rate = 0.01， num_epochs = 15， batch_size = 16
# 最后将模型权重保存到指定文件夹

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader, Subset
import random
import numpy as np
import copy
import os
from torch.utils.tensorboard import SummaryWriter

# 设置随机种子以确保结果的可重复性
random.seed(42)
torch.manual_seed(42)
np.random.seed(42)

# 数据路径
data_dir = 'D:/FDU/graduate/研一下/神网/期中作业/CUB_200_2011/CUB_200_2011/images'

# 定义数据转换
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# 加载整个数据集
full_dataset = datasets.ImageFolder(data_dir, transform=data_transforms['train'])

# 获取类别索引
class_indices = {cls: [] for cls in full_dataset.classes}

# 按类别收集图像索引
for idx, (path, class_idx) in enumerate(full_dataset.imgs):
    class_indices[full_dataset.classes[class_idx]].append(idx)

# 划分训练集和验证集
train_indices = []
val_indices = []

for cls, indices in class_indices.items():
    random.shuffle(indices)
    split = int(0.8 * len(indices))  # 80% 训练，20% 验证
    train_indices.extend(indices[:split])
    val_indices.extend(indices[split:])

# 创建训练集和验证集
train_dataset = Subset(full_dataset, train_indices)
val_dataset = Subset(full_dataset, val_indices)

# 应用不同的transform
train_dataset.dataset.transform = data_transforms['train']
val_dataset.dataset.transform = data_transforms['val']

# 创建DataLoader
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 输出一些信息以确认
print(f"Total images: {len(full_dataset)}")
print(f"Training images: {len(train_dataset)}")
print(f"Validation images: {len(val_dataset)}")

# 设置 device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 定义损失函数
criterion = nn.CrossEntropyLoss()

# 训练和验证函数
def train_model(model, criterion, optimizer, dataloaders, device, num_epochs=15, save_path='best_model.pth'):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    writer = SummaryWriter()  # 初始化TensorBoard writer

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            # 记录到TensorBoard
            if phase == 'train':
                writer.add_scalar('Loss/train', epoch_loss, epoch)
                writer.add_scalar('Accuracy/train', epoch_acc, epoch)
            else:
                writer.add_scalar('Loss/val', epoch_loss, epoch)
                writer.add_scalar('Accuracy/val', epoch_acc, epoch)
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    model.load_state_dict(best_model_wts)
    torch.save(model.state_dict(), save_path)  # 保存最佳模型权重
    writer.close()  # 关闭TensorBoard writer
    return model

# 最优超参数
learning_rate = 0.01
num_epochs = 15
batch_size = 16
save_path = 'best_model.pth'

# 重新初始化数据加载器
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
dataloaders = {'train': train_loader, 'val': val_loader}

# 重新初始化模型和优化器
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 200)
model = model.to(device)

optimizer = optim.SGD([
    {'params': model.fc.parameters(), 'lr': learning_rate},
    {'params': [param for name, param in model.named_parameters() if "fc" not in name], 'lr': learning_rate * 0.1}
], momentum=0.9, weight_decay=1e-4)

print(f'Learning rate: {learning_rate}, Epochs: {num_epochs}, Batch size: {batch_size}')
model = train_model(model, criterion, optimizer, dataloaders, device, num_epochs, save_path)

print(f"Training completed. Best model saved to {save_path}.")


## 实验结果
* 训练过程中在训练集和验证集上的loss曲线和验证集上的accuracy变化如下图所示，其中loss曲线图如下：奇数列为训练集；偶数列为验证集；且参数依次为：learning rate=0.01，epoch=10，batch size=16；learning rate=0.01，epoch=10，batch size=32；learning rate=0.01，epoch=15，batch size=16；learning rate=0.01，epoch=15，batch size=32；learning rate=0.01，epoch=20，batch size=16；learning rate=0.01，epoch=20，batch size=32；learning rate=0.001，epoch=10，batch size=16；learning rate=0.001，epoch=10，batch size=32；learning rate=0.001，epoch=15，batch size=16；learning rate=0.001，epoch=15，batch size=32；learning rate=0.001，epoch=20，batch size=16；learning rate=0.001，epoch=20，batch size=32。

<img src="./Train_Phase_0_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_0_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_1_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_1_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_2_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_2_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_3_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_3_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_4_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_4_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_5_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_5_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_6_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_6_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_7_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_7_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_8_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_8_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_9_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_9_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_10_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_10_loss.svg" alt="验证集loss曲线" width="700">
<img src="./Train_Phase_11_loss.svg" alt="训练集loss曲线" width="700">
<img src="./Validation_Phase_11_loss.svg" alt="验证集loss曲线" width="700">

* 经预训练的验证集Accuracy曲线图如下；参数分布同上。

<img src="./Validation_Phase_0_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_1_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_2_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_3_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_4_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_5_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_6_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_7_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_8_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_9_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_10_accuracy.svg" alt="验证集Acc曲线" width="700">
<img src="./Validation_Phase_11_accuracy.svg" alt="验证集Acc曲线" width="700">

* 选择最优参数即LR=0.01,Batch size=16，Epoch=15，进行预训练的模型loss曲线图和Acc曲线图如下：

<img src="./Loss_train.svg" alt="验证集Acc曲线" width="700">
<img src="./Loss_val.svg" alt="验证集Acc曲线" width="700">
<img src="./Accuracy_train.svg" alt="验证集Acc曲线" width="700">
<img src="./Accuracy_val.svg" alt="验证集Acc曲线" width="700">

* 下图为未预训练的模型的loss曲线图和Acc曲线图，可以看出未预训练的模型的准确度相比预训练的模型要低得多。

<img src="./loss.svg" alt="未预训练的模型的loss曲线" width="700">
<img src="./accuracy.svg" alt="未预训练的模型的Acc曲线" width="700">



# 通过上面分析可以得出以下结论：
# 经过预训练的模型可以在训练数据上训练5轮后达到收敛效果，在验证集准确率可以达到0.78
# 随机初始化的模型会在40轮之后达到收敛，收敛后在验证集的最高准确率仅有0.29左右
# 可以看出是否经过预训练对最后的结果是非常巨大的